A web scrapper for pulling information off of competitors websites

In [1]:
## Ben Kite github code

from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

Loding in the definitions put together by the Ben Kite github code

In [2]:
def findTables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

In [3]:
def pullTable(url, tableID):
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    data_rows = tables[0].findAll('tr')
    data_header = tables[0].findAll('thead')
    data_header = data_header[0].findAll("tr")
    data_header = data_header[0].findAll("th")
    game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pandas.DataFrame(game_data)
    header = []
    for i in range(len(data.columns)):
        header.append(data_header[i].getText())
    data.columns = header
    data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    return(data)

In [4]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://www.whitelabs.com/yeast-single?id=102&type=YEAST'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [5]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'class':'container'})
gaz_data

[<div class="main-nagivation-desktop container">
 <div class="main-nagivation-desktop__wrapper--row row">
 <div class="col-lg-1">
 <a href="https://www.whitelabs.com" class="main-logo">
 <img src="https://www.whitelabs.com/public/uploads/system_setting_images/WL_Web_Header_Seal_Logo-1609882734.svg" alt="White Labs Logo">
 </a>
 </div>
 <div class="col-lg-11">
 <nav class="navbar-bar navbar-bar__top" aria-label="primary navigation">
 <ul class="navbar-bar__wrapper">
 <li class="nav-item">
 <a class="nav-link dropdown-toggle" href="https://www.whitelabs.com/about-white-labs">About Us </a>
 <ul class="sub-menu">
 
 <li> <a href="https://www.whitelabs.com/about-white-labs">
 About Us
 </a></li>
 <li> <a href="https://www.whitelabs.com/team">
 team
 </a></li>
 <li> <a class="nav-link" href="https://www.whitelabs.com/career">
 careers
 </a></li>
 </ul>
 </li>
 <li class="nav-item">
 <a class="nav-link" href="https://www.whitelabs.com/media">
 media
 </a>
 </li>
 <li class="nav-item">
 <a cla

In [6]:
# narrow things down to the table
gaz_table = soup.find('div', {'class':'accordion-style'})
gaz_table

<div class="accordion-style" id="accordionEx" role="tablist" aria-multiselectable="true">
  <ul class="beer_style">
    <li>
American IPA
</li>
    <li>
Blonde Ale
</li>
    <li>
Brown Ale
</li>
    <li>
Double IPA
</li>
    <li>
English Bitter
</li>
    <li>
English IPA
</li>
    <li>
Hazy/Juicy IPA
</li>
    <li>
Imperial Stout
</li>
    <li>
Old Ale
</li>
    <li>
Pale Ale
</li>
    <li>
Porter
</li>
    <li>
Red Ale
</li>
    <li>
Stout
</li>
  </ul>
</div>

In [27]:
gaz_table = soup.find('div', {'class':'accordion-style'})
gaz_table

a = (str(gaz_table.html))
a = a.replace('<li>\n' , '')
a = a.replace('\n</li>\n', '')
x = len(a)
print(x)
a = a[120:]
print(a)

309
American IPA    Blonde Ale    Brown Ale    Double IPA    English Bitter    English IPA    Hazy/Juicy IPA    Imperial Stout    Old Ale    Pale Ale    Porter    Red Ale    Stout  </ul>
</div>


In [8]:
style_list = a.split()
style_list = style_list[:-2]

#for word in a.split():
    #style_list.append(a)
    #print(word)
    

In [9]:
print(style_list)

['American', 'IPA', 'Blonde', 'Ale', 'Brown', 'Ale', 'Double', 'IPA', 'English', 'Bitter', 'English', 'IPA', 'Hazy/Juicy', 'IPA', 'Imperial', 'Stout', 'Old', 'Ale', 'Pale', 'Ale', 'Porter', 'Red', 'Ale', 'Stout']


In [10]:
# use find() to parse a specific HTML
gaz_data = soup.find('main', {'class':'main-content'})
gaz_data

<main class="main-content">
<section class="single-product-content default-section">
<div class="container">
<div class="row">
<div class="col-md-12 text-left">
<a href="https://www.whitelabs.com/yeast-bank" class="btn btn--back"> Back to Yeast Bank </a>
</div>
<div class="col-md-12 single-product-content__header">
<h1>English Ale Yeast</h1>
<div class="single-product-content__badge">
<ul class="inline-block badge-icon">
<li class="badge-icon">
<img class="classYeast" alt="Yeast: Ale" src="https://www.whitelabs.com/public/uploads/banner_images/strain2-1603762141.png">
</li>
</ul>
<ul class="inline-block badge-icon">
<li class="badge-icon">
<img class="classYeast" alt="Distilling" src="https://www.whitelabs.com/public/uploads/banner_images/strain1-1600398080-1603762224.png">
</li>
</ul>
<ul class="inline-block badge-icon">
<li class="badge-icon">
<img class="classYeast" alt="Organic Strains" src="https://www.whitelabs.com/public/uploads/banner_images/WL_Cat-Organic_Icon-1607709325.png">

In [23]:
strain_id = soup.find('/h3')
print(strain_id)

None


In [11]:
# use find() to parse a specific HTML
metrics = soup.find('p')
metrics

[<p>63%
 -
 70%</p>,
 <p/>,
 <p>Very High</p>,
 <p>Medium (5-10%)</p>,
 <p> 65°
 -
 68° F
 <br>18°
 -
 20° C
 </p>,
 <p>Negative
 </p>,
 <p>This is a classic ESB strain from one of England’s largest independent breweries. While it is traditionally used for English-style ales including milds, bitters, porters, and stouts, it is also ideal for American-style pale ales and IPAs. Residual sweetness accentuates malt character along with mild fruity esters, adding complexity to the flavor and aroma of finished beers. Slight diacetyl production is common. Due to this strain’s high flocculation, the beer will finish clear and the yeast can easily be harvested from the fermenter for future use. It is common for this yeast to look coagulated.<br>
 <br>
 <strong><font color="#000000"><a href="https://yeastman.com/calculator" target="_blank">Pitch Rate Calculator</a></font></strong><br>
  
 <p data-pm-slice="1 1 []"><strong><a href="https://whitelabsbrewingco.com/beer-archive/" target="_blank">Cur

In [12]:
import re

clean_metrics =[]

for i in metrics:
    i = str(i).replace('</p>', '')
    i = str(i).replace('<p>', '')
    #i = str(i).replace("\\r\\n", '')
    i = re.sub('\s+', '', i)
    print('new')
    print(i)
    clean_metrics.append(i)


new
63%-70%
new
<p/>
new
VeryHigh
new
Medium(5-10%)
new
65°-68°F<br>18°-20°C
new
Negative
new
ThisisaclassicESBstrainfromoneofEngland’slargestindependentbreweries.WhileitistraditionallyusedforEnglish-stylealesincludingmilds,bitters,porters,andstouts,itisalsoidealforAmerican-stylepalealesandIPAs.Residualsweetnessaccentuatesmaltcharacteralongwithmildfruityesters,addingcomplexitytotheflavorandaromaoffinishedbeers.Slightdiacetylproductioniscommon.Duetothisstrain’shighflocculation,thebeerwillfinishclearandtheyeastcaneasilybeharvestedfromthefermenterforfutureuse.Itiscommonforthisyeasttolookcoagulated.<br><br><strong><fontcolor="#000000"><ahref="https://yeastman.com/calculator"target="_blank">PitchRateCalculator</a></font></strong><br><pdata-pm-slice="11[]"><strong><ahref="https://whitelabsbrewingco.com/beer-archive/"target="_blank">Curiousaboutbrewingwiththisstrain?WhiteLabsBrewingCohasthedatayouneed!</a></strong>
new
<pclass="footer-title">Sales&CustomerService
new
<pclass="footer-title">Co

In [13]:
attenuation = clean_metrics[0]
flocculation = clean_metrics[2]
alcohol_tolerance = clean_metrics[3]
fermentation_temperature = clean_metrics[4]
STA1 = clean_metrics[5]
description = metrics[6]

print(attenuation)
print(flocculation)
print(alcohol_tolerance)
print(fermentation_temperature)
print(STA1)
print(description)

63%-70%
VeryHigh
Medium(5-10%)
65°-68°F<br>18°-20°C
Negative
<p>This is a classic ESB strain from one of England’s largest independent breweries. While it is traditionally used for English-style ales including milds, bitters, porters, and stouts, it is also ideal for American-style pale ales and IPAs. Residual sweetness accentuates malt character along with mild fruity esters, adding complexity to the flavor and aroma of finished beers. Slight diacetyl production is common. Due to this strain’s high flocculation, the beer will finish clear and the yeast can easily be harvested from the fermenter for future use. It is common for this yeast to look coagulated.<br>
<br>
<strong><font color="#000000"><a href="https://yeastman.com/calculator" target="_blank">Pitch Rate Calculator</a></font></strong><br>
 
<p data-pm-slice="1 1 []"><strong><a href="https://whitelabsbrewingco.com/beer-archive/" target="_blank">Curious about brewing with this strain? White Labs Brewing Co has the data you need

In [14]:
# use find() to parse a specific HTML
strain = soup.find('h1')
strain = str(strain).replace('<h1>', '')
strain = str(strain).replace('</h1>', '')
strain

'English Ale Yeast'

In [15]:
import pandas as pd

entry = {'Strain': [strain],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [alcohol_tolerance],
        'Fermentation_Temperature' : [fermentation_temperature],
        'STA1' : [STA1],
        'Description' : [description]}


df = pd.DataFrame(entry)

df

,Strain,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,English Ale Yeast,63%-70%,VeryHigh,Medium(5-10%),65°-68°F<br>18°-20°C,Negative,<p>This is a classic ESB strain from one of En...


# Pulling all the urls that are needed

In [25]:
url_list = []
# https://www.whitelabs.com/yeast-single?id=101&type=YEAST&style_type=0

url_1 = 'https://www.whitelabs.com/yeast-single?id='
url_2 = '&type=YEAST&style_type=0'

i = 100

while i < 400:
    full_url = url_1 + str(i) + url_2
    url_list.append(full_url)
    i = i+1
    #print(full_url)

https://www.whitelabs.com/yeast-single?id=100&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=101&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=102&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=103&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=104&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=105&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=106&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=107&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=108&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=109&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=110&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=111&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=112&type=YEAST&style_type=0
https://www.whitelabs.com/yeast-single?id=113&type=YEAST&style_type=0
https://www.whitelab

In [26]:
print(len(url_list))

300
